In [6]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import random

from google.cloud import storage

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import datasets

from keras.layers import Input, Lambda, Dense, Flatten, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

import keras_tuner
from kerastuner.tuners import RandomSearch

from numpy import savez_compressed
from numpy import load

from datetime import datetime

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [7]:
(X_train, y_train), (X_test,y_test) = datasets.cifar10.load_data()

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.2)

In [9]:
y_train = y_train.reshape(-1,)
y_test = y_test.reshape(-1,)
y_val = y_val.reshape(-1,)

In [10]:
X_train = X_train / 255.0
X_test = X_test / 255.0
X_val = X_val / 255.0

In [11]:
#sample
sample_size= 100

sample_i_train = random.sample(range(len(X_train)), round(sample_size*0.6))
X_train = X_train[sample_i_train]
y_train = y_train[sample_i_train]

sample_i_test = random.sample(range(len(X_test)), round(sample_size*0.2))
X_test = X_test[sample_i_test]
y_test = y_test[sample_i_test]

sample_i_val = random.sample(range(len(X_val)), round(sample_size*0.2))
X_val = X_val[sample_i_val]
y_val = y_val[sample_i_val]

In [ ]:
#https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/
#https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiE0eGZ_qL0AhVrg_0HHebcC6kQFnoECAQQAw&url=https%3A%2F%2Fsicara.ai%2Fblog%2F2019-10-31-convolutional-layer-convolution-kernel&usg=AOvVaw3p8dCzNohorImGouz_x-tg
#https://medium.com/analytics-vidhya/how-to-choose-the-size-of-the-convolution-filter-or-kernel-size-for-cnn-86a55a1e2d15

In [22]:
def build_cnn(hp):
    ####Config####
    input_pixels = 32
    input_depth = 3
    num_outputs = 10
    ####Config####

    model = tf.keras.Sequential()

    num_inputs = input_pixels
    num_depth = input_depth

    
    chc_filters_direction = hp.Choice('chc_filters_direction', [0.5, 1.0, 2.0])

    if chc_filters_direction == 0.5:
        num_filters = hp.Choice('num_filters_init', [128, 256, 512])
    elif chc_filters_direction == 1.0:
        num_filters = hp.Choice('num_filters_init', [32, 64, 128, 256, 512])
    elif chc_filters_direction == 2.0:
        num_filters = hp.Choice('num_filters_init', [32, 64, 128])

    num_kernels = hp.Choice('num_filters_init', [3, 5]) 
    rat_cdropout = hp.Choice('conv_dropout', [0.0,0.1,0.2])

    for b in range(hp.Int('num_blocks', 1, 3)):
        for c in range(hp.Int('num_clayers', 1, 3)):
            model.add(tf.keras.layers.Conv2D(filters= num_filters,
                                            kernel_size= (num_kernels,num_kernels),
                                            strides=(1, 1),
                                            padding="same",
                                            activation='relu', 
                                            input_shape = (num_inputs, num_inputs,num_depth)))

            num_inputs = (num_inputs-num_kernels+1)/1 +1 
            num_depth = num_filters #todo - drauf achten, dass es nicht zu groß wird.

            model.add(tf.keras.layers.Dropout(rate = rat_cdropout))
        
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), # var idr nur 3 oder 2 <- todo
                                            strides = (2,2), #idr immer 2 https://cs231n.github.io/convolutional-networks/#norm
                                            padding = "same")) 
        num_inputs = (num_inputs-2)/2 +1
        num_filters = num_filters * chc_filters_direction
                                            
    model.add(tf.keras.layers.Flatten()) #<- todo ob ggf. noch andere Alternative

    for d in range(hp.Int('num_dlayers', 1, 3)):
        num_units = hp.Int('num_units_' + str(d),min_value=32,max_value=512,step=32)
        model.add(tf.keras.layers.Dense(units = num_units, 
                                        activation='relu'))
        model.add(tf.keras.layers.Dropout(rate = 0.5)) #todo var

    model.add(tf.keras.layers.Dense(units=num_outputs,
                                    activation='softmax'))  

    model.compile(
        optimizer = "adam",
        loss = 'sparse_categorical_crossentropy',
        metrics=['accuracy', 'sparse_categorical_crossentropy'])

    model.summary()


    return model

In [23]:
tuner = RandomSearch(
    build_cnn,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='tuner',
    project_name='cifar_test')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        98336     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                262176    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [24]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
num_filters_init (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128, 256, 512], 'ordered': True}
chc_filters_direction (Choice)
{'default': 0.5, 'conditions': [], 'values': [0.5, 1.0, 2.0], 'ordered': True}
conv_dropout (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.1, 0.2], 'ordered': True}
num_blocks (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
num_clayers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
num_dlayers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
num_units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [25]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_val, y_val))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_filters_init  |512               |?                 
chc_filters_dir...|2                 |?                 
conv_dropout      |0.1               |?                 
num_blocks        |2                 |?                 
num_clayers       |2                 |?                 
num_dlayers       |1                 |?                 
num_units_0       |480               |?                 

